In [ ]:
import json

import pandas as pd #библиотека для анализа табличных данных
pd.set_option('display.max_columns', 100) # ограничение на вывод таблиц - количество колонок
pd.set_option('display.max_rows', 100) # ограничение на вывод таблиц - количество строк

## Импорт данных

In [ ]:
with open("auto-crashes.json") as data_file:    
    data = json.load(data_file)

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data)
df = pd.concat([df.drop(['infoDtp'], axis=1), df['infoDtp'].apply(pd.Series)], axis=1) 

df['datetime'] = df[['date', 'Time']].apply(lambda x: ' '.join(x), axis=1)
df['datetime'] = pd.to_datetime(df['datetime'])

df['week_day'] = df['datetime'].apply(lambda x: x.strftime('%A'))
week_day_dict = {'Monday':"1-Monday", 'Tuesday':"2-Tuesday", 'Wednesday':"3-Wednesday", 'Thursday':"4-Thursday", 'Friday':"5-Friday", 'Saturday':"6-Saturday", 'Sunday':"7-Sunday"} 
weekend_day_dict = {'Monday':"Workday", 'Tuesday':"Workday", 'Wednesday':"Workday", 'Thursday':"Workday", 'Friday':"Workday", 'Saturday':"Weekend", 'Sunday':"Weekend"} 
df['weekend'] = df['week_day'].map(weekend_day_dict)
df['week_day'] = df['week_day'].map(week_day_dict)
df['day'] = df['datetime'].apply(lambda x: x.strftime('%d'))
df['hour'] = df['datetime'].apply(lambda x: x.strftime('%H'))
df['month'] = df['datetime'].apply(lambda x: x.strftime('%m'))
df['year'] = df['datetime'].apply(lambda x: x.strftime('%Y'))
df['n'] = 1

In [ ]:
df

## Анализ табличных данных

ДТП по типу

In [ ]:
df['DTP_V'].value_counts()

ДТП по улицам

In [ ]:
df['street'].value_counts()

ДТП по годам

In [ ]:
df['year'].value_counts()

ДТП по типу/смертности

In [ ]:
pd.DataFrame(df.pivot_table(aggfunc=sum, values="POG", columns = "DTP_V")).sort_values("POG",ascending=False)

## Анализ иерархичных данных

Выделяем ДТП с определенными нарушениями - пьянка

In [ ]:
violation_types = set()

for dtp in data:
    for ts_info in dtp['infoDtp']['ts_info']:
        for ts_uch in ts_info['ts_uch']: 
            for violation in (ts_uch['NPDD'] + ts_uch['SOP_NPDD']):
                violation_types.add(violation)
                
violation_types

In [ ]:
def check_drunk_drivers(row):
    for ts_info in row['ts_info']:
        for ts_uch in ts_info['ts_uch']: 
            if ts_uch['K_UCH'] == "Водитель":
                for violation in (ts_uch['NPDD'] + ts_uch['SOP_NPDD']):
                    if "опьянения" in violation:
                        return True
                    
    return False
    
    
df['drunk_drivers'] = df.apply(check_drunk_drivers, axis=1)

In [ ]:
df["drunk_drivers"].value_counts()

In [ ]:
pd.DataFrame(df.pivot_table(aggfunc=sum, values="POG", columns = "drunk_drivers")).sort_values("POG",ascending=False)

Выделяем долю водителей без ОСАГО по годам

In [ ]:
def check_osago_drivers(row):
    osago_count = 0
    for ts_info in row['ts_info']:
        for ts_uch in ts_info['ts_uch']: 
            if ts_uch['K_UCH'] == "Водитель":
                for violation in (ts_uch['NPDD'] + ts_uch['SOP_NPDD']):
                    if "ОСАГО" in violation:
                        osago_count = osago_count + 1
            
    return osago_count
    
    
df['osago_drivers'] = df.apply(check_osago_drivers, axis=1)

In [ ]:
kts = df.pivot_table(columns=["year"], aggfunc=sum, values="K_TS")
kts_no_osago = df.pivot_table(columns=["year"], aggfunc=sum,values="osago_drivers")
osago_drivers = pd.concat([kts, kts_no_osago], axis=1)
osago_drivers['normalize'] = osago_drivers['osago_drivers']/osago_drivers['K_TS']
osago_drivers